In [2]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [3]:
from tqdm.keras import TqdmCallback

c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
retriever_location_ = r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_06_07_22\retriever_port_v2_hoo" #r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_05_27\retriever_port_v2"
ranking_location_ = r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_05_27\tf_listwise_ranking_2024_05_27_11_20"
stock_info_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\stock_data.xlsx"

test_ds_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_hoo_test"
train_ds_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_hoo_train"

portfolios_loc = r"D:/dev work/recommender systems/Atrad_CARS/data/portfolios_v2/portfolios"

In [5]:
retriever_location_ = r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_06_10_15\retriever_port_v2" #r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_05_27\retriever_port_v2"
ranking_location_ = r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_05_27\tf_listwise_ranking_2024_05_27_11_20"
stock_info_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\stock_data.xlsx"

test_ds_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_test"
train_ds_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_train"

portfolios_loc = r"D:/dev work/recommender systems/Atrad_CARS/data/portfolios_v2/portfolios"

In [6]:
test_ds = tf.data.Dataset.load(test_ds_loc).cache()

train_ds = tf.data.Dataset.load(train_ds_loc).cache()

train_ds_1 = train_ds.batch(len(train_ds))
test_ds_1 = test_ds.batch(len(test_ds))

portfolios = tf.data.Dataset.load(portfolios_loc).cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])

unique_item_ids = np.unique(np.concatenate(list(items_ids)))
unique_item_names = np.unique(np.concatenate(list(item_names)))
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# need these to initialize timestamp embedding layers in future steps

timestamps = np.concatenate(list(portfolios.map(lambda x: x["UNIX_TS"]).batch(100)))
max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

In [7]:
train_ds_dict = next(iter(train_ds_1))
test_ds_dict = next(iter(test_ds_1))
#test
data_dict = {
    'CDSACCNO': test_ds_dict['CDSACCNO'].numpy(),
    'STOCKCODE': test_ds_dict['STOCKCODE'].numpy(),
    'STOCKNAME': test_ds_dict['STOCKNAME'].numpy(),
    'GICS': test_ds_dict['GICS'].numpy(),
    'UNIX_TS': test_ds_dict['UNIX_TS'].numpy(),
    'RATING': test_ds_dict['RATING'].numpy(),
}

test_df = pd.DataFrame.from_dict(data_dict)

test_df = test_df.astype(
    {
        'CDSACCNO' : 'str',
        'STOCKCODE' : 'str',
        'STOCKNAME' : 'str',
        'GICS' : 'str'
        })

#train
data_dict = {
    'CDSACCNO': train_ds_dict['CDSACCNO'].numpy(),
    'STOCKCODE': train_ds_dict['STOCKCODE'].numpy(),
    'STOCKNAME': train_ds_dict['STOCKNAME'].numpy(),
    'GICS': train_ds_dict['GICS'].numpy(),
    'UNIX_TS': train_ds_dict['UNIX_TS'].numpy(),
    'RATING': train_ds_dict['RATING'].numpy(),
}

train_df = pd.DataFrame.from_dict(data_dict)

train_df = train_df.astype(
    {
        'CDSACCNO' : 'str',
        'STOCKCODE' : 'str',
        'STOCKNAME' : 'str',
        'GICS' : 'str'
        })

In [8]:
from retrieval_recommender import Retriever

retriever = Retriever(
    use_timestamp = True,
    portfolios = portfolios
)

retriever.load_weights(retriever_location_)

retriever.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


In [9]:
from ranker_recommender import Ranker

ranker = Ranker(
    loss = tf.keras.losses.MeanSquaredError(),
    portfolios = portfolios
)

ranker.load_weights(ranking_location_)
ranker.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


In [10]:
stock_info = pd.read_excel(stock_info_loc)
stock_info = stock_info.drop(['Unnamed: 0','buisnesssummary'],axis = 1)
stock_info = stock_info.rename(columns = {
    'symbol':'STOCKCODE',
    'name' : 'STOCKNAME',
    'gics_code' : 'GICS'
})
stock_info = stock_info[~stock_info['GICS'].isna()]

stock_info.shape
print("items data shape :: {}".format(stock_info.shape))

items_ds = tf.data.Dataset.from_tensor_slices(stock_info.to_dict(orient= 'list'))

items data shape :: (280, 3)


In [11]:
code2name = dict(zip(stock_info.STOCKCODE, stock_info.STOCKNAME))
code2gics = dict(zip(stock_info.STOCKCODE, stock_info.GICS))

# complete recommender function

In [12]:
items_identiifiers = items_ds.map(lambda x: x["STOCKCODE"])
items_identiifiers = next(iter(items_identiifiers.batch(len(items_identiifiers))))
items_identiifiers.shape

TensorShape([280])

In [13]:
index = tfrs.layers.factorized_top_k.BruteForce(
    query_model = retriever.user_model,
    k = 10)
    
retriever_item_model = retriever.item_model
mapped_items = items_ds.batch(len(items_ds)).map(lambda x : retriever_item_model(x, map_ = True))

mapped_items_tensor = next(iter(mapped_items))
index.index(mapped_items_tensor, items_identiifiers)

In [14]:
test_users_ = test_df.groupby('CDSACCNO')
train_users_ = train_df.groupby('CDSACCNO')

In [15]:
def recommend_(CDSACCNO, timestamp = 1664821800.0):

    seen_items = train_df.loc[train_df.CDSACCNO == CDSACCNO].STOCKCODE.values.reshape(1, -1)

    
    _, recommendations = index.query_with_exclusions(
    queries = (
        tf.constant([CDSACCNO]),
        tf.constant([timestamp])
        ),
    exclusions = seen_items
    )

    recommendations = [reco.decode('utf-8') for reco in recommendations.numpy().flatten()]
    # print(f"Recommendations for user %s: {recommendations}" %(test_user))

    names = np.array([code2name[code] for code in recommendations])
    gics = np.array([code2gics[code] for code in recommendations])

    user = {
    'CDSACCNO' : np.array([CDSACCNO]),
    'STOCKCODE' : np.array(recommendations).reshape(-1,10),
    'GICS' : gics.reshape(-1,10),
    'STOCKNAME' : names.reshape(-1,10)
    }

    pred_ratings = ranker(user)

    recommendations_w_ratings = pd.DataFrame()
    recommendations_w_ratings['STOCKCODE'] = recommendations
    recommendations_w_ratings['PRED_RATING'] = pred_ratings.numpy().flatten()
    recommendations_w_ratings = recommendations_w_ratings.sort_values( by = ['PRED_RATING'], ascending= False).reset_index(drop = True)

    user_test_port_ = test_df.iloc[test_users_.groups[CDSACCNO]].sort_values('RATING', ascending = False)
    recommendations_w_ratings = recommendations_w_ratings.join(user_test_port_.set_index('STOCKCODE'), on = 'STOCKCODE')[['STOCKCODE','PRED_RATING','RATING']]
    
    hit_perc = (10 - recommendations_w_ratings['RATING'].isna().sum())/len(recommendations_w_ratings)

    return recommendations_w_ratings, hit_perc


In [16]:
test_timestamp = datetime.timestamp(datetime.now())

total_hit_perc_ = 0
results_ = pd.DataFrame(columns = ['CDSACCNO','hit_perc'])

for user in tqdm(test_df.CDSACCNO.unique()):
    _, hit_perc_ = recommend_(user, test_timestamp)
    results_ = pd.concat(
        [
            results_,
        pd.DataFrame([
            {
                'CDSACCNO':user,
                'hit_perc':hit_perc_
                }
            ])
        ], ignore_index = True)
    total_hit_perc_ += hit_perc_

# (total_hit_perc_/test_df.CDSACCNO.nunique())*100
total_hit_perc_

  0%|          | 0/5906 [00:00<?, ?it/s]C:\Users\naradaw\AppData\Local\Temp\ipykernel_8264\2312754527.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_ = pd.concat(
  0%|          | 6/5906 [00:04<31:27,  3.13it/s]  

  0%|          | 7/5906 [00:04<28:13,  3.48it/s]

100%|██████████| 5906/5906 [05:46<00:00, 17.04it/s]


292.1999999999977

In [17]:
(total_hit_perc_/test_df.CDSACCNO.nunique())*100

4.947511005756819

In [18]:
results_

,CDSACCNO,hit_perc
0,HDF-74565-LI/00,0.1
1,BMS-800262640-VN/00,0.0
2,COM-69742-LC/00,0.1
3,BMS-861802000-VN/00,0.3
4,HDF-743463188-VN/00,0.0
...,...,...
5901,BMS-68660-LI/00,0.0
5902,BMS-683600342-VN/00,0.0
5903,CAS-86452-LI/00,0.1
5904,CMB-5826-LC/00,0.0


In [19]:
results_.hit_perc.value_counts()

hit_perc
0.0    3653
0.1    1724
0.2     426
0.3      79
0.4      17
0.5       4
0.7       1
0.8       1
0.6       1
Name: count, dtype: int64

In [20]:
# results_df_ = pd.DataFrame()

# results_df_['hit_perc'] = results_

# Precision and Recall @k

In [21]:
import array
import collections

from typing import Dict, List, Optional, Text, Tuple

def evaluate(retriever,
             test: tf.data.Dataset,
             train: Optional[tf.data.Dataset] = None,
             timestamp: int = datetime.timestamp(datetime.now()),
             k: int = 10) -> Dict[Text, float]:

  item_ids = stock_info['STOCKCODE'].unique()
  
  item_ids = np.concatenate(
      list(items_ds.batch(1000).map(lambda x: x["STOCKCODE"]).as_numpy_iterator()))

  item_vocabulary = dict(zip(item_ids.tolist(), range(len(item_ids))))

  train_user_to_items = collections.defaultdict(lambda: array.array("i"))
  test_user_to_items = collections.defaultdict(lambda: array.array("i"))

  if train is not None:
    for row in train.as_numpy_iterator():
      user_id = row["CDSACCNO"]
      item_id = item_vocabulary[row["STOCKCODE"]]
      train_user_to_items[user_id].append(item_id)

  for row in test.as_numpy_iterator():
    user_id = row["CDSACCNO"]
    item_id = item_vocabulary[row["STOCKCODE"]]
    test_user_to_items[user_id].append(item_id)

  item_embeddings = mapped_items_tensor.numpy()

  user_ids = []
  precision_values = []
  recall_values = []
  num_test_items = []
  num_train_items = []

  for user_id, test_items in tqdm(test_user_to_items.items()):
    user_embedding = retriever.user_model(
      (
        tf.constant([user_id]),
        tf.constant([timestamp])
       )
      ).numpy()
    scores = (user_embedding @ item_embeddings.T).flatten()

    test_items = np.frombuffer(test_items, dtype=np.int32)
    
    if train is not None:
      train_items = np.frombuffer(
          train_user_to_items[user_id], dtype=np.int32)
      scores[train_items] = -1e6

    

    top_items = np.argsort(-scores)[:k]
    num_test_items_in_k = sum(x in top_items for x in  test_items)
    precision_values.append(num_test_items_in_k / k)
    recall_values.append(num_test_items_in_k / len(test_items))
    num_test_items.append(len((test_items)))
    num_train_items.append(len(train_user_to_items[user_id]))
    user_ids.append(user_id)

  results_df_ = pd.DataFrame(
    columns = ['CDSACCNO','precision@k', 'recall@k','num_test_items','portfolio_size'],
    data = list(zip(user_ids, precision_values, recall_values, num_test_items, num_train_items))
  )

  return {
      "precision_at_k": np.mean(precision_values),
      "recall_at_k": np.mean(recall_values),
      "results_df_" : results_df_
  }

In [22]:
results = evaluate(
    retriever,
    test_ds,
    train_ds
)

100%|██████████| 5906/5906 [00:41<00:00, 143.02it/s]


In [24]:
results['results_df_']
# results

,CDSACCNO,precision@k,recall@k,num_test_items,portfolio_size
0,b'HDF-74565-LI/00',0.1,0.500000,2,9
1,b'BMS-800262640-VN/00',0.0,0.000000,2,10
2,b'COM-69742-LC/00',0.1,0.100000,10,38
3,b'BMS-861802000-VN/00',0.3,0.214286,14,58
4,b'HDF-743463188-VN/00',0.0,0.000000,2,9
...,...,...,...,...,...
5901,b'BMS-68660-LI/00',0.0,0.000000,3,10
5902,b'BMS-683600342-VN/00',0.0,0.000000,3,10
5903,b'CAS-86452-LI/00',0.1,0.333333,3,10
5904,b'CMB-5826-LC/00',0.0,0.000000,2,8


# Hit Ratio - HOO

In [32]:
def recommend_hoo_(CDSACCNO, timestamp = 1664821800.0):

    seen_items = train_df.loc[train_df.CDSACCNO == CDSACCNO].STOCKCODE.values.reshape(1, -1)

    
    _, recommendations = index.query_with_exclusions(
    queries = (
        tf.constant([CDSACCNO]),
        tf.constant([timestamp])
        ),
    exclusions = seen_items
    )

    recommendations = [reco.decode('utf-8') for reco in recommendations.numpy().flatten()]
    # print(f"Recommendations for user %s: {recommendations}" %(test_user))

    names = np.array([code2name[code] for code in recommendations])
    gics = np.array([code2gics[code] for code in recommendations])

    user = {
    'CDSACCNO' : np.array([CDSACCNO]),
    'STOCKCODE' : np.array(recommendations).reshape(-1,10),
    'GICS' : gics.reshape(-1,10),
    'STOCKNAME' : names.reshape(-1,10)
    }

    pred_ratings = ranker(user)

    recommendations_w_ratings = pd.DataFrame()
    recommendations_w_ratings['STOCKCODE'] = recommendations
    recommendations_w_ratings['PRED_RATING'] = pred_ratings.numpy().flatten()
    recommendations_w_ratings = recommendations_w_ratings.sort_values( by = ['PRED_RATING'], ascending= False).reset_index(drop = True)

    user_test_port_ = test_df.iloc[test_users_.groups[CDSACCNO]].sort_values('RATING', ascending = False)
    recommendations_w_ratings = recommendations_w_ratings.join(user_test_port_.set_index('STOCKCODE'), on = 'STOCKCODE')[['STOCKCODE','PRED_RATING','RATING']]
    
    hit_ = 1 if (recommendations_w_ratings.RATING.isna().all()) == False else 0

    return recommendations_w_ratings, hit_


In [33]:
from tqdm import tqdm
from datetime import datetime

test_timestamp = datetime.timestamp(datetime.now())

hits_ = 0
for user in tqdm(test_df.CDSACCNO.unique()):
    _, hit_ = recommend_hoo_(user, test_timestamp)
    hits_ += hit_

  0%|          | 0/5906 [00:00<?, ?it/s]

100%|██████████| 5906/5906 [07:15<00:00, 13.55it/s]


In [34]:
(hits_/test_df.CDSACCNO.nunique())*100

38.14764646122587

epochs - 11.03962072468676
hoo - 11.395191330849983
